In [1]:

import os
import cv2
import time
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Activation, Dropout

In [2]:
img_size = 100
datadir = r'F:\cvpr\Dataset'
CATEGORIES = os.listdir(datadir)
print(CATEGORIES)

['mask_weared_incorrect', 'without_mask', 'with_mask']


In [3]:
# Define two empty list for contain image data
x, y = [], []
   
def PreProcess():
    for category in CATEGORIES:
        path = os.path.join(datadir, category)
        classIndex = CATEGORIES.index(category)
        print(path)
        for imgs in tqdm(os.listdir(path)):
            img_arr = cv2.imread(os.path.join(path, imgs))
            
            # resize the image
            resized_array = cv2.resize(img_arr, (img_size, img_size))
            cv2.imshow("images", resized_array)
            cv2.waitKey(1)
            resized_array = resized_array/255.0
            x.append(resized_array)
            y.append(classIndex)
            
PreProcess()
cv2.destroyAllWindows()

F:\cvpr\Dataset\mask_weared_incorrect


100%|██████████████████████████████████████████████████████████████████████████████| 2994/2994 [00:46<00:00, 64.12it/s]


F:\cvpr\Dataset\without_mask


100%|██████████████████████████████████████████████████████████████████████████████| 2994/2994 [00:46<00:00, 64.26it/s]


F:\cvpr\Dataset\with_mask


100%|██████████████████████████████████████████████████████████████████████████████| 2994/2994 [00:54<00:00, 55.33it/s]


In [4]:
# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [5]:
# Convert and resize the data to a numpy array
X_train = np.array(X_train).reshape(-1, img_size, img_size, 3)
y_train = np.array(y_train)
X_test = np.array(X_test).reshape(-1, img_size, img_size, 3)
y_test = np.array(y_test)

In [6]:
batch_size = 32
epochs = 15

In [7]:
# Create the model architecture

model = Sequential()

model.add(Conv2D(64,(3, 3), input_shape=(img_size, img_size, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(16, activation='relu'))


model.add(Dense(len(CATEGORIES)))
model.add(Activation('softmax'))

# compile the model

model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 64)        1792      
                                                                 
 activation (Activation)     (None, 98, 98, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 256)       147712    
                                                                 
 activation_1 (Activation)   (None, 47, 47, 256)       0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 256)      0         
 2D)                                                    

In [8]:
t1 = time.time()

# fit the model
model.fit(X_train, y_train, batch_size = batch_size, epochs=5, validation_split=0.3, verbose = 1)
model.save('{}.h5'.format("model"))

t2 = time.time()
print('Time taken: ',t2-t1)

Epoch 1/5
158/158 [==============================] - 234s 1s/step - loss: 0.5862 - accuracy: 0.7487 - val_loss: 0.3545 - val_accuracy: 0.8678
Epoch 2/5
158/158 [==============================] - 249s 2s/step - loss: 0.3246 - accuracy: 0.8861 - val_loss: 0.2627 - val_accuracy: 0.9119
Epoch 3/5
158/158 [==============================] - 208s 1s/step - loss: 0.2514 - accuracy: 0.9073 - val_loss: 0.2705 - val_accuracy: 0.8970
Epoch 4/5
158/158 [==============================] - 209s 1s/step - loss: 0.2074 - accuracy: 0.9296 - val_loss: 0.1827 - val_accuracy: 0.9295
Epoch 5/5
158/158 [==============================] - 207s 1s/step - loss: 0.1783 - accuracy: 0.9364 - val_loss: 0.1549 - val_accuracy: 0.9411
Time taken:  1110.2753925323486


In [9]:
validation_loss, validation_accuracy = model.evaluate(X_test, y_test)

57/57 [==============================] - 12s 214ms/step - loss: 0.1595 - accuracy: 0.9438


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the model
model = load_model('model.h5')

# Define mediapipe Face detector

face_detection = mp.solutions.face_detection.FaceDetection()

# Detection function



def get_detection(frame):

    height, width, channel = frame.shape

    # Convert frame BGR to RGB colorspace

    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Detect results from the frame
    
    result = face_detection.process(imgRGB)


    try:
        for count, detection in enumerate(result.detections):

            # print(detection)

        
            # Extract bounding box information 
            
            box = detection.location_data.relative_bounding_box

            x, y, w, h = int(box.xmin*width), int(box.ymin * height), int(box.width*width), int(box.height*height)
            
    # If detection is not available then pass 
    except:
        pass

    return x, y, w, h

CATEGORIES = ['no_mask', 'mask']
cap = cv2.VideoCapture(0)
while True:
    _, frame = cap.read()
    img = frame.copy()
    try:
        x, y, w, h = get_detection(frame)
        
        crop_img = img[y:y+h, x:x+w]
        
        crop_img = cv2.resize(crop_img, (100, 100))
        
        crop_img = np.expand_dims(crop_img, axis=0)
        
        # get the prediction from the model.
        prediction = model.predict(crop_img)
        print(prediction)
        index = np.argmax(prediction)
        res = CATEGORIES[index]
        if index == 0:
            color = (0, 0, 255)
        else:
            color = (0, 255, 0)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, res, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX,
                                 0.8, color, 2, cv2.LINE_AA)

    except:
        pass
    
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 370ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 34ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 33ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 35ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 27ms/step
[[3.5313791e-22 1.0000000e+00 1.0915191e-37]]
1/1 [==============================] - 0s 37ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 29ms/step
[[6.0467080e-24 1.0000000e+00 6.4930414e-38]]
1/1 [==============================] - 0s 32ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 27ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 26ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 39ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 39ms/step
[[8.1063862e-21 1.0000000e+00 9.3073264e-20]]
1/1 [==============================] - 0s 62ms/step
[[1.0562868e-23 1.0000000e+00 1.2946667e-26]]
1/1 [=================

1/1 [==============================] - 0s 51ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 50ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 48ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 52ms/step
[[0.0000000e+00 1.0000000e+00 1.3643216e-38]]
1/1 [==============================] - 0s 59ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 50ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 59ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 54ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 49ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 52ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 50ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 52ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 58ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 50ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 63ms/step
[[0. 

[[0. 1. 0.]]
1/1 [==============================] - 0s 40ms/step
[[8.3254525e-35 1.0000000e+00 0.0000000e+00]]
1/1 [==============================] - 0s 52ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 67ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 59ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 53ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 48ms/step
[[0.000000e+00 1.000000e+00 4.374153e-28]]
1/1 [==============================] - 0s 55ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 57ms/step
[[0.0000000e+00 1.0000000e+00 3.4056017e-38]]
1/1 [==============================] - 0s 68ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 31ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 28ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 58ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 65ms/step
[[0.0000000e+00 1.0000000e+00 8.8699267e-38]]
1/1 [========

1/1 [==============================] - 0s 57ms/step
[[2.6457595e-29 1.8199691e-08 1.0000000e+00]]
1/1 [==============================] - 0s 64ms/step
[[9.999999e-01 8.754020e-31 8.645239e-08]]
1/1 [==============================] - 0s 56ms/step
[[0.0000000e+00 1.0000000e+00 5.2937226e-25]]
1/1 [==============================] - 0s 51ms/step
[[0.0000000e+00 1.0000000e+00 2.1926201e-26]]
1/1 [==============================] - 0s 71ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 59ms/step
[[1.4497967e-12 1.0000000e+00 1.9736686e-14]]
1/1 [==============================] - 0s 52ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 67ms/step
[[0.000000e+00 1.000000e+00 7.010092e-20]]
1/1 [==============================] - 0s 63ms/step
[[2.4168827e-19 1.0000000e+00 3.2697191e-22]]
1/1 [==============================] - 0s 50ms/step
[[0.0000000e+00 1.0000000e+00 1.5337069e-32]]
1/1 [==============================] - 0s 64ms/step
[[3.3872865e-20 1.0000000e+00 1.6946535e